In [6]:

from utils import get_device
from model_task_3 import train_model
from data_processing import get_loaders, class_cols
from model_task_3 import create_model, train_model
import torch

import pandas as pd
from tqdm import tqdm

In [ ]:

device = get_device()

train_loader, val_loader, train_df, val_df = get_loaders(
    image_size=(384, 384),
    num_workers=0,
)
class_cols_order = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"]
num_classes = len(class_cols_order)
model = create_model(num_classes=num_classes, model_name="vit_base_patch16_384")


class_counts = [int(train_df[c].sum()) for c in class_cols_order]

#base_model = create_model(num_classes=num_classes, model_name="vit_base_patch16_384")
#ckpt = torch.load("best_model.pth", map_location=device)
#base_model.load_state_dict(ckpt["model_state"])


model, history2 = train_model(
    train_loader,
    val_loader,
    num_classes=num_classes,
    device=device,
    epochs=15,                 
    class_counts=class_counts,
    #model=base_model,          
    save_path="best_model.pth" 
)

Epoch 1/30 train_loss=0.2618 val_loss=2.9973 train_acc=0.0497 val_acc=0.0592
  ✓ New best val_acc=0.0592, model saved to best_model.pth


Epoch 2/30 train_loss=0.1565 val_loss=3.4493 train_acc=0.0707 val_acc=0.0206


Epoch 3/30 train_loss=0.1337 val_loss=2.5317 train_acc=0.0996 val_acc=0.1184
  ✓ New best val_acc=0.1184, model saved to best_model.pth


Epoch 4/30 train_loss=0.1256 val_loss=2.7537 train_acc=0.1017 val_acc=0.0732


Epoch 5/30 train_loss=0.1192 val_loss=2.4248 train_acc=0.1074 val_acc=0.1337
  ✓ New best val_acc=0.1337, model saved to best_model.pth


...

Epoch 15/30 train_loss=0.0815 val_loss=2.0767 train_acc=0.2162 val_acc=0.2974


Epoch 16/30 train_loss=0.0752 val_loss=2.2914 train_acc=0.2676 val_acc=0.2735


Epoch 17/30 train_loss=0.0696 val_loss=1.9433 train_acc=0.2788 val_acc=0.2954


Epoch 18/30 train_loss=0.0610 val_loss=2.1399 train_acc=0.2750 val_acc=0.2861


KeyboardInterrupt: 